In [6]:
from langchain.output_parsers import ResponseSchema
from langchain.output_parsers import StructuredOutputParser



In [23]:
#criando Schema


schema_presente = ResponseSchema(
    name='presente',
    type='boolean',
    description='O item foi comprado para alguém? True caso verdadeiro e False caso falso ou não tenha informação' 
)


schema_entrega = ResponseSchema(
    name='dia_entrega',
    type='int',
    description='Quantos dias para a entrega chegar?  Se a resposta não for encontrada, retorne -1' 
)


schema_valor = ResponseSchema(
    name='percerpcao_de_valor',
    type='list',
    description='Extraia qualquer frase sobre o valor do produto. Retorne uma lista de python' 
)

In [24]:
Response_schema = [schema_presente, schema_entrega, schema_valor]

In [25]:
output_parser = StructuredOutputParser.from_response_schemas(Response_schema)
schema_formated = output_parser.get_format_instructions()
print(schema_formated)

The output should be a markdown code snippet formatted in the following schema, including the leading and trailing "```json" and "```":

```json
{
	"presente": boolean  // O item foi comprado para alguém? True caso verdadeiro e False caso falso ou não tenha informação
	"dia_entrega": int  // Quantos dias para a entrega chegar?  Se a resposta não for encontrada, retorne -1
	"percerpcao_de_valor": list  // Extraia qualquer frase sobre o valor do produto. Retorne uma lista de python
}
```


In [30]:
from langchain.prompts import  ChatPromptTemplate

review_cliente="""um soprador foi comprado, chegou em 2 dias, foi presente para minha esposa, excelete pois é economico e eficiente"""


review_template = ChatPromptTemplate.from_template(
    """
      Para o texto a seguir, responda as seguintes perguntas:

      presente, dias_entrega, percepcao_de_valor

      Texto: {review}

      {schema}
  """, partial_variables={'schema': schema_formated})



In [31]:
print(review_template.format_messages(review=review_cliente))

[HumanMessage(content='\n      Para o texto a seguir, responda as seguintes perguntas:\n\n      presente, dias_entrega, percepcao_de_valor\n\n      Texto: um soprador foi comprado, chegou em 2 dias, foi presente para minha esposa, excelete pois é economico e eficiente\n\n      The output should be a markdown code snippet formatted in the following schema, including the leading and trailing "```json" and "```":\n\n```json\n{\n\t"presente": boolean  // O item foi comprado para alguém? True caso verdadeiro e False caso falso ou não tenha informação\n\t"dia_entrega": int  // Quantos dias para a entrega chegar?  Se a resposta não for encontrada, retorne -1\n\t"percerpcao_de_valor": list  // Extraia qualquer frase sobre o valor do produto. Retorne uma lista de python\n}\n```\n  ', additional_kwargs={}, response_metadata={})]


In [35]:
from langchain_openai.chat_models import ChatOpenAI

chat = ChatOpenAI()
resposta = chat.invoke(review_template.format_messages(review=review_cliente))


In [36]:
print(resposta.content)

```json
{
	"presente": true,
	"dia_entrega": 2,
	"percerpcao_de_valor": ["excelete pois é economico e eficiente"]
}
```


In [37]:
r_json = output_parser.parse(resposta.content)
print(r_json)

{'presente': True, 'dia_entrega': 2, 'percerpcao_de_valor': ['excelete pois é economico e eficiente']}


In [38]:
r_json['presente']

True

In [39]:
r_json['dia_entrega']

2

In [40]:
r_json['percerpcao_de_valor']

['excelete pois é economico e eficiente']